# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [3]:
#Your code here
import pandas as pd

# Load data
df = pd.read_csv("homepage_actions.csv")

# Preview
print(df.shape)
df.head()




(8188, 4)


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [4]:
print(df['group'].value_counts())
print(df['action'].value_counts())

group
control       4264
experiment    3924
Name: count, dtype: int64
action
view     6328
click    1860
Name: count, dtype: int64


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [5]:
#Your code here
# --- A/B Test: Experiment vs Control (Clicks given Views) ---

import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

# If df isn't already loaded, uncomment:
# df = pd.read_csv("homepage_actions.csv")

# 1) Build per-user action table to ensure one view/click per user
#    (unique user ids for each action per group)
group_action = (
    df.pivot_table(index="group", columns="action", values="id",
                   aggfunc="nunique", fill_value=0)
)

# Total viewers per group (denominator)
group_action["total"] = group_action["view"]

# Extract counts
control_clicks = int(group_action.loc["control", "click"])
control_total  = int(group_action.loc["control", "total"])
exp_clicks     = int(group_action.loc["experiment", "click"])
exp_total      = int(group_action.loc["experiment", "total"])

# 2) Conversion rates
control_rate = control_clicks / control_total
exp_rate     = exp_clicks / exp_total

# 3) Two-proportion z-test (one-tailed: experiment > control)
count = np.array([exp_clicks, control_clicks])
nobs  = np.array([exp_total,  control_total])
z_stat, p_value_one_tailed = proportions_ztest(count, nobs, alternative="larger")

# (Optional) two-tailed p-value if you need it:
_, p_value_two_tailed = proportions_ztest(count, nobs, alternative="two-sided")

# 4) 95% confidence intervals for each group’s conversion rate (Wilson)
ci_exp_low, ci_exp_high = proportion_confint(exp_clicks, exp_total, alpha=0.05, method="wilson")
ci_ctl_low, ci_ctl_high = proportion_confint(control_clicks, control_total, alpha=0.05, method="wilson")

# 5) Pretty print
print("=== A/B Test: Homepage Conversion ===")
print(f"Control:    {control_clicks}/{control_total}  (CTR = {control_rate:.4%}) "
      f"[95% CI {ci_ctl_low:.4%}, {ci_ctl_high:.4%}]")
print(f"Experiment: {exp_clicks}/{exp_total}  (CTR = {exp_rate:.4%}) "
      f"[95% CI {ci_exp_low:.4%}, {ci_exp_high:.4%}]")
print()
print(f"Z-statistic (one-tailed exp>ctl): {z_stat:.4f}")
print(f"p-value (one-tailed):             {p_value_one_tailed:.6f}")
print(f"p-value (two-tailed, optional):   {p_value_two_tailed:.6f}")

alpha = 0.05
decision = "REJECT H0 (experiment > control)" if p_value_one_tailed < alpha else "FAIL TO REJECT H0"
print(f"\nDecision at α={alpha}: {decision}")


=== A/B Test: Homepage Conversion ===
Control:    932/3332  (CTR = 27.9712%) [95% CI 26.4732%, 29.5200%]
Experiment: 928/2996  (CTR = 30.9746%) [95% CI 29.3442%, 32.6538%]

Z-statistic (one-tailed exp>ctl): 2.6186
p-value (one-tailed):             0.004415
p-value (two-tailed, optional):   0.008830

Decision at α=0.05: REJECT H0 (experiment > control)


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [6]:
#Your code here

# Step 1: Expected clicks for experiment group if same CTR as control

# Control group data
control_clicks = 932
control_views = 3332

# Experiment group data
exp_views = 2996

# Control CTR
control_ctr = control_clicks / control_views

# Expected clicks in experiment if same CTR as control
expected_exp_clicks = exp_views * control_ctr

print("Control CTR:", control_ctr)
print("Expected experiment clicks (if same CTR as control):", expected_exp_clicks)


Control CTR: 0.2797118847539016
Expected experiment clicks (if same CTR as control): 838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [7]:
#Your code here
import math

# Given data
control_clicks = 932
control_views = 3332
exp_clicks = 928
exp_views = 2996

# Step 1: Control CTR and expected clicks in experiment
control_ctr = control_clicks / control_views
expected_exp_clicks = exp_views * control_ctr

# Step 2: Standard deviation under H0
variance = exp_views * control_ctr * (1 - control_ctr)
std_dev = math.sqrt(variance)

# Number of standard deviations (z-score without p-value yet)
std_devs_away = (exp_clicks - expected_exp_clicks) / std_dev

print("Expected clicks (H0):", expected_exp_clicks)
print("Observed clicks:", exp_clicks)
print("Standard deviation:", std_dev)
print("Standard deviations away (z-score):", std_devs_away)


Expected clicks (H0): 838.0168067226891
Observed clicks: 928
Standard deviation: 24.568547907005815
Standard deviations away (z-score): 3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [8]:
#Your code here

from scipy.stats import norm

# Using z-score from Step 2
z_score = std_devs_away   # from Step 2

# One-tailed p-value (experiment > control)
p_value_one_tailed = 1 - norm.cdf(z_score)

# Two-tailed p-value (if needed)
p_value_two_tailed = 2 * (1 - norm.cdf(abs(z_score)))

print("Z-score:", z_score)
print("One-tailed p-value:", p_value_one_tailed)
print("Two-tailed p-value:", p_value_two_tailed)


Z-score: 3.6625360854823588
One-tailed p-value: 0.00012486528006949715
Two-tailed p-value: 0.0002497305601389943


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: Yes — the manual test and the z-test give slightly different numbers, but both show the same conclusion.
The experiment homepage’s conversion rate is significantly higher than the control’s.
Therefore, we reject the null hypothesis and confirm the new design is more effective

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.